In [1]:
# Requires spark >= 2.3
# Requires PyArrow (pip install pyarrow)

In [2]:
import sys
import os

In [3]:
pt = "/usr/local/opt/python/bin/python3.7"
java_location = "/Library/Java/JavaVirtualMachines/adoptopenjdk-8.jdk/Contents/Home"
#jl2 = /Contents/Home

In [4]:
os.environ['PYSPARK_PYTHON'] = pt
os.environ['PYSPARK_DRIVER_PYTHON'] = pt
os.environ['JAVA_HOME'] = java_location

In [5]:
print(sys.executable)

/usr/local/opt/python/bin/python3.7


In [6]:
from pyspark import SparkContext, SQLContext

import pyspark.sql.types as T
import pyspark.sql.functions as F

from pyspark.sql.functions import pandas_udf
from pyspark.sql.functions import PandasUDFType

In [7]:
sc = SparkContext()
sqlc = SQLContext(sc)

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.net.BindException: Can't assign requested address: Service 'sparkDriver' failed after 16 retries (on a random free port)! Consider explicitly setting the appropriate binding address for the service 'sparkDriver' (for example spark.driver.bindAddress for SparkDriver) to the correct binding address.
	at sun.nio.ch.Net.bind0(Native Method)
	at sun.nio.ch.Net.bind(Net.java:433)
	at sun.nio.ch.Net.bind(Net.java:425)
	at sun.nio.ch.ServerSocketChannelImpl.bind(ServerSocketChannelImpl.java:223)
	at io.netty.channel.socket.nio.NioServerSocketChannel.doBind(NioServerSocketChannel.java:128)
	at io.netty.channel.AbstractChannel$AbstractUnsafe.bind(AbstractChannel.java:558)
	at io.netty.channel.DefaultChannelPipeline$HeadContext.bind(DefaultChannelPipeline.java:1283)
	at io.netty.channel.AbstractChannelHandlerContext.invokeBind(AbstractChannelHandlerContext.java:501)
	at io.netty.channel.Abstract

In [ ]:
schema = T.StructType(
    [
        T.StructField('group_col', T.StringType()),
        T.StructField('column1', T.IntegerType()),
        T.StructField('column2', T.IntegerType())
    ])

df = sqlc.createDataFrame(
    [
        ['a', 1, 1], 
        ['c', 3, 4], 
        ['e', 5, 6],
        ['a', 2, 7],
        ['a', 3, 2],
        ['e', 3, 6]
    ],
    schema=schema)

df.toPandas()

In [ ]:
# make schema for the return dataframe of the udf
udf_schema = T.StructType(df.schema.fields + [T.StructField('size', T.IntegerType())])

# define a function that inputs a dataframe (corresponding to grouped data)
# and outputs a dataframe with the specified data
# In this case, the function appends the size of the grouped dataframe as a column.

@pandas_udf(udf_schema, PandasUDFType.GROUPED_MAP)
def add_count(grouped_df):
    return grouped_df.assign(size=len(grouped_df))

In [ ]:
df.groupBy('group_col').apply(add_count).toPandas()